In [2]:
import pandas as pd
import Translation_1
import warnings

In [21]:
data_1=pd.read_csv('Isshop.csv')

In [24]:
Words_to_search = [
"Proceed to Payment",
"Place Order",
"Place your order"
"Shopping Basket"
"View Cart",
"Order Now",
"Purchase",
"Shop Now",
"Complete Purchase",
"Add to Basket",
"Buy Now",
"Continue Shopping",
"Secure Checkout",
"Confirm Order",
"Order Summary",
"Add to Wishlist",
"Select product",
"Return and exchange policy",
"Track your order",
"Shipping & Payment",
"Online shop",
"Exchanges & Refunds",
"How To Shop",
"Order Status",
"Shipping Method",
"Tracking Order",
"Online E-Commerce",
"Quick Buy",
"Easy Return Guidance",
"Shipping Details",
"Delivery",
"Returns And Exchange", 
"Payment Methods",
"Payment method",
"Returns And Refunds",
"Shipping Information & Costs",
"Shopping Basket",
"Shipping And Payment",
"Returns / Product Exchange",
"Payment", 
"Shipping & Delivery",
"Exchanges & Returns",
"Shipping And Exchange Policy",
"Refund Policy",
"All About Shopping",
"Basket",
"Shopping Cart",
"Return Procedure",
"Guarantees And Returns",
"Return/Complaint Instructions",
"E-Shop",
"Order Status ",
"Our Stores",
"How to Order",
"Eshop",
"Free shipping",
"How to Ship",
"Add to cart",
"Out of stock",
"Delivery Information",
"Returns Information",
"my cart",
"My basket",
]


______

In [25]:
# Initialize translation cache
translation_cache = {}

def find_phrases_optimized(row, phrases_to_search):
    text_lang = row['detected_language']
    text = row['HTML']
    
    if not isinstance(text, str):
        return ''
    
    found_phrases = []
    text_lower = text.lower()

    for phrase in phrases_to_search:
        if phrase.lower() in text.lower():
            found_phrases.append(phrase)
    if text_lang != 'en':      
 
        # Check/create language cache if needed
        if text_lang not in translation_cache:
            translation_cache[text_lang] = {}
        
        # Search with translation
        for phrase in phrases_to_search:
            # Get from cache or translate
            if phrase not in translation_cache[text_lang]:
                try:
                    translation_cache[text_lang][phrase] = Translation_1.api_translate_text('en', text_lang, phrase).lower()
                except:
                    translation_cache[text_lang][phrase] = None
            
            translated_phrase = translation_cache[text_lang][phrase]
            if translated_phrase and translated_phrase in text_lower:
                found_phrases.append(phrase)
    
    return ", ".join(found_phrases)

# Apply to your dataframe
data_1['words_found'] = data_1.apply(
    lambda row: find_phrases_optimized(row, Words_to_search), 
    axis=1
)

generate two data frames df_1 with language, df_2 with detected
 
join the results.

clean the dataset.

In [3]:
df_1=pd.read_csv("language.csv")
df_2=pd.read_csv("detected.csv")

In [7]:
warnings.filterwarnings(action="ignore")
df_1["words_found"] = df_1["words_found"].apply(
    lambda x: ', '.join(pd.unique([i.strip() for i in x.split(",")]).tolist()) if isinstance(x, str) else ""
)
df_2["words_found"] = df_1["words_found"].apply(
    lambda x: ', '.join(pd.unique([i.strip() for i in x.split(",")]).tolist()) if isinstance(x, str) else ""
)


In [10]:
combined=df_2.merge(df_1[["domain","words_found"]], how="left", on="domain")

In [12]:
combined["words_found_x"].fillna("",inplace=True)
combined["words_found_y"].fillna("", inplace=True)

In [13]:
combined["word_count_x"]=combined["words_found_x"].apply(lambda x: len([i.strip() for i in x.split(",")])if isinstance(x, str) else 0)
combined["word_count_y"]=combined["words_found_y"].apply(lambda x: len([i.strip() for i in x.split(",")])if isinstance(x, str) else 0)

In [15]:
combined.head(3)

,domain,HTML,Text,found_phrases,language,detected_language,words_found_x,words_found_y,word_count_x,word_count_y
0,esgotado.com,"\n<!doctype html>\n<html class=""no-js"" lang=""e...",Link ALternatif JOKER5000 Situs JOKER Deposit ...,"Add to cart, add to cart, Payment",en,en,"Purchase, Payment, Add to cart","Purchase, Payment, Add to cart",3,3
1,esh2u.com,"<!doctype html>\n<html class=""no-js"" lang=""en""...",Best Electrical Appliances Online – ESH Electr...,"Privacy policy, Add to cart, add to cart, Deli...",en,en,"Purchase, Continue Shopping, Delivery, Payment...","Purchase, Continue Shopping, Delivery, Payment...",9,9
2,nykdbynykaa.com,"<!doctype html>\n<html class=""js"" lang=""en"">\n...",Nykd By Nykaa – Nykd by Nykaa Skip to content ...,"Terms and conditions, Privacy policy, Return a...",en,en,"Purchase, Shop Now, Buy Now, Continue Shopping...","Purchase, Shop Now, Buy Now, Continue Shopping...",12,12


In [16]:
combined["words_found"] = combined["words_found_x"].where(
    combined["word_count_x"] >= combined["word_count_y"],
    combined["words_found_y"]
)

In [17]:
combined.drop(columns=["found_phrases", "words_found_x",	"words_found_y", "word_count_x", "word_count_y"], inplace=True)

In [18]:
combined.rename(columns={"Text":"HTML_text"})
combined.fillna('',inplace=True)